In [1]:
#Muhammad Shafwan Faturrahman (13316006)
#2. Transfer Learning and Without Augmentation

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import tensorflow as tf  
import tensorflow.keras as keras  # pakai Keras dari tensorflow
import os  
from tensorflow.keras.layers import Flatten, Dense, AveragePooling2D, GlobalAveragePooling2D  
from tensorflow.keras.models import Model  
from tensorflow.keras.optimizers import RMSprop, SGD  
from tensorflow.keras.callbacks import ModelCheckpoint  
from tensorflow.keras.callbacks import EarlyStopping  
from tensorflow.keras.preprocessing.image import ImageDataGenerator  
from tensorflow.keras.callbacks import CSVLogger  
from tensorflow.keras.layers import BatchNormalization  
from tensorflow.keras.models import load_model  
import numpy as np  
from pathlib import Path  
import shutil

In [0]:
#making training & validation directories  
import pathlib  
session='/content/drive/My Drive/Colab Notebooks/SiscerSTI/Tugas3/TL_NoAug/TL_NoAug'  
train_dir='/content/drive/My Drive/Colab Notebooks/SiscerSTI/Tugas3/train_validation_split/train'  
valid_dir='/content/drive/My Drive/Colab Notebooks/SiscerSTI/Tugas3/train_validation_split/valid'  
classnames=['Black-grass','Charlock','Cleavers','Common Chickweed','Common wheat','Fat Hen','Loose Silky-bent','Maize','Scentless Mayweed','Shepherds Purse','Small-flowered Cranesbill','Sugar beet']


In [0]:
#prepare model    
img_width=224
img_height=224
cnn_notop=keras.applications.nasnet.NASNetMobile(input_shape=(img_width, img_height, 3)
                                                 , include_top=False
                                                 , weights='imagenet' # menggunakan transfer learning
                                                 #, weights=None # tanpa transfer learning
                                                 , input_tensor=None
                                                 , pooling=None
                                                )

In [0]:
x = cnn_notop.output  
x = GlobalAveragePooling2D()(x)      
x = Dense(1024, activation='relu')(x)      
x = BatchNormalization()(x)  
predictions = Dense(12, activation='softmax')(x)  
the_model = Model(cnn_notop.input, predictions)

In [6]:
#training  
learning_rate = 0.0001   
logfile = session + '-train' + '.log'   
batch_size=32
nbr_epochs=100
print("training  directory: "+train_dir)  
print("valication directory: "+valid_dir)  
optimizer = SGD(lr=learning_rate, momentum=0.9, decay=0.0, nesterov=True)  
the_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])  
csv_logger = CSVLogger(logfile, append=True)  
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')  
best_model_filename=session+'-weights.{epoch:02d}-{val_loss:.2f}.h5'   
best_model = ModelCheckpoint(best_model_filename, monitor='val_accuracy', verbose=1, save_best_only=True)  
  # this is the augmentation configuration we will use for training  
train_datagen = ImageDataGenerator(  
    rescale=1. / 255,  
    #shear_range=0.2,  
    #zoom_range=0.2,  
    #rotation_range=0,  
    #width_shift_range=0.2,  
    #height_shift_range=0.2,  
    horizontal_flip=False,  
    vertical_flip=False)  
val_datagen = ImageDataGenerator(rescale=1. / 255)  
print('prepare train generator')   
train_generator = train_datagen.flow_from_directory(  
    train_dir,  
    target_size=(img_width, img_height),  
    batch_size=batch_size,  
    shuffle=True,  
    classes=classnames,  
    class_mode='categorical')  
print('prepare validation generator')   
validation_generator = val_datagen.flow_from_directory(  
    valid_dir,  
    target_size=(img_width, img_height),  
    batch_size=batch_size,  
    shuffle=True,  
    classes=classnames,  
    class_mode='categorical')

training  directory: /content/drive/My Drive/Colab Notebooks/SiscerSTI/Tugas3/train_validation_split/train
valication directory: /content/drive/My Drive/Colab Notebooks/SiscerSTI/Tugas3/train_validation_split/valid
prepare train generator
Found 3812 images belonging to 12 classes.
prepare validation generator
Found 954 images belonging to 12 classes.


In [7]:
print('fit generator')   
the_model.fit(  
    train_generator,  
    steps_per_epoch = train_generator.samples//batch_size,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples//batch_size,
    epochs=nbr_epochs,  
    verbose=1,  
    callbacks=[best_model, csv_logger, early_stopping]
)

fit generator
Epoch 1/100
119/119 [==============================] - ETA: 0s - loss: 2.4276 - accuracy: 0.2587
Epoch 00001: val_accuracy improved from -inf to 0.16918, saving model to /content/drive/My Drive/Colab Notebooks/SiscerSTI/Tugas3/TL_NoAug/TL_NoAug-weights.01-2.41.h5
119/119 [==============================] - 310s 3s/step - loss: 2.4276 - accuracy: 0.2587 - val_loss: 2.4062 - val_accuracy: 0.1692
Epoch 2/100
119/119 [==============================] - ETA: 0s - loss: 1.3571 - accuracy: 0.5733
Epoch 00002: val_accuracy improved from 0.16918 to 0.32651, saving model to /content/drive/My Drive/Colab Notebooks/SiscerSTI/Tugas3/TL_NoAug/TL_NoAug-weights.02-2.11.h5
119/119 [==============================] - 89s 749ms/step - loss: 1.3571 - accuracy: 0.5733 - val_loss: 2.1061 - val_accuracy: 0.3265
Epoch 3/100
119/119 [==============================] - ETA: 0s - loss: 0.9102 - accuracy: 0.7212
Epoch 00003: val_accuracy improved from 0.32651 to 0.37177, saving model to /content/drive/M

In [0]:
#prediction   
batch_size=4  
nbr_test_samples=794    
#choose weights file manually   
#weights_path = 'simpleNASNet-weights.10-0.17.h5' # choose file manually, filename may be different  
#weights_path = 'sesi-01-weights.08-0.58.h5'
weights_path = session+'-weights.85-0.34.h5'
test_data_dir ='/content/drive/My Drive/Colab Notebooks/SiscerSTI/Tugas3/test-224x224/'

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)   
test_generator = test_datagen.flow_from_directory(   
    directory=test_data_dir,   
    target_size=(img_width, img_height),   
    batch_size=batch_size,   
    shuffle = False, # no shuffling, since filenames must match predictions. Shuffling may change file sequence   
    classes = None, #    
    class_mode = None)   
test_image_list = test_generator.filenames   
print('Loading model and weights')   
predict_model = load_model(weights_path)   
#predict_model = the_model

Found 794 images belonging to 1 classes.
Loading model and weights


In [10]:
train_generator = train_datagen.flow_from_directory(  
    train_dir,  
    target_size=(img_width, img_height),  
    batch_size=batch_size,  
    shuffle=True,  
    classes=classnames,  
    class_mode='categorical',
    subset='training') 

Found 3812 images belonging to 12 classes.


In [11]:
predict_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 111, 111, 32) 128         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           stem_bn1[0][0]                   
______________________________________________________________________________________________

In [12]:
print('Begin to predict for testing data ...')   
predictions = predict_model.predict(x=test_generator, steps=nbr_test_samples/batch_size,verbose=1)   
np.savetxt(session+'-predictions.txt', predictions) # store prediction matrix, for later analysis if necessary

Begin to predict for testing data ...
199/198 [==============================] - 567s 3s/step


In [13]:
#submission  
submission_file=session+'-submit.csv'   
print('Begin to write submission file:'+submission_file)   
f_submit = open(submission_file, 'w')   
f_submit.write('file,species\n')   
for i, image_name in enumerate(test_image_list):   
  # find maximum prediction of 12  
  max_index=0  
  max_value=0  
  for x in range(0, 12):  
    if(predictions[i][x]>max_value):  
      max_value=predictions[i][x]  
      max_index=x  
  basename=os.path.basename(image_name)   
  prediction_class = classnames[max_index] # get predictions from array     
  f_submit.write('%s,%s\n' % (basename, prediction_class))   
f_submit.close()   
print('Finished write submission file ..')

Begin to write submission file:/content/drive/My Drive/Colab Notebooks/SiscerSTI/Tugas3/TL_NoAug/TL_NoAug-submit.csv
Finished write submission file ..
